In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install recordlinkage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.9/926.9 kB 17.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 18.5 MB/s eta 0:00:00


In [3]:
import recordlinkage

from recordlinkage.datasets import load_febrl1, load_febrl2

# load_febrl1: 1000 records (500 originals and 500 duplicates, with exactly one duplicate per original record).
# load_febrl2: 5000 records (4000 originals and 1000 duplicates), with a maximum of 5 duplicates per one original record.

In [4]:
df_train, y_train = load_febrl1(return_links=True)  # DataFrame to train (1000 records)
df_test, y_test = load_febrl2(return_links=True)  # DataFrame to test (5000 records)

In [5]:
indexer = recordlinkage.Index().full()  # Initiate indexer
candidate_links_train = indexer.index(df_train)  # Index pairs of records for the training set
candidate_links_test = indexer.index(df_test)  # Index pairs of records for the testing set

In [6]:
df_train.describe()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
count,956,982,955,975,885,982,1000,985,959,1000
unique,378,494,154,618,585,520,451,17,513,550
top,joshua,white,1,badimara street,rowethorpe,blacktown,2614,nsw,19581231,6988048
freq,21,22,35,6,13,10,8,353,4,2


In [7]:
df_test.describe()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
count,4891,4936,4777,4891,4431,4950,5000,4952,4890,5000
unique,907,1816,397,2455,2517,1684,1412,19,3785,4089
top,emiily,white,1,forbes street,rosetta village,toowoomba,4701,nsw,19920917,1135598
freq,85,131,142,16,40,46,23,1604,7,6


In [8]:
comp = recordlinkage.Compare()

comp.string(
    "given_name",
    "given_name",
    method="jarowinkler",
    label="given_name"
)
comp.string(
    "surname",
    "surname",
    method="jarowinkler",
    label="surname"
)
comp.string("street_number", "street_number", label="street_number", threshold=0.85)
comp.string("address_1", "address_1", label="address_1", threshold=0.85)
comp.string("address_2", "address_2", label="address_2", threshold=0.85)
comp.string("suburb", "suburb", label="suburb", threshold=0.85)
comp.string("postcode", "postcode", label="postcode", threshold=0.85)
comp.string("state", "state", label="state", threshold=0.85)
comp.string("date_of_birth", "date_of_birth", label="date_of_birth", threshold=0.85)
comp.exact("soc_sec_id", "soc_sec_id", label="soc_sec_id")

<Compare>

In [9]:
features_train = comp.compute(candidate_links_train, df_train)

In [10]:
features_train

given_name   surname  street_number  address_1  \
rec_id_1     rec_id_2                                                        
rec-122-org  rec-223-org      0.000000  0.455556            0.0        0.0   
rec-373-org  rec-223-org      0.000000  0.511111            0.0        0.0   
             rec-122-org      0.539683  0.533333            0.0        0.0   
rec-10-dup-0 rec-223-org      0.000000  0.511111            0.0        0.0   
             rec-122-org      0.676190  0.533333            0.0        0.0   
...                                ...       ...            ...        ...   
rec-212-org  rec-327-org      0.000000  0.490079            0.0        0.0   
             rec-188-dup-0    0.000000  0.492063            0.0        0.0   
             rec-334-dup-0    0.000000  0.000000            0.0        0.0   
             rec-469-dup-0    0.000000  0.417989            0.0        0.0   
             rec-350-dup-0    0.000000  0.523810            0.0        0.0   

                            address_2  suburb  postcode  state  date_of_birth  \
rec_id_1     rec_id_2                                                           
rec-122-org  rec-223-org          0.0     0.0       0.0    0.0            0.0   
rec-373-org  rec-223-org          0.0     0.0       0.0    0.0            0.0   
             rec-122-org          0.0     0.0       0.0    0.0            0.0   
rec-10-dup-0 rec-223-org          0.0     0.0       0.0    0.0            0.0   
             rec-122-org          0.0     0.0       0.0    0.0            0.0   
...                               ...     ...       ...    ...            ...   
rec-212-org  rec-327-org          0.0     0.0       0.0    0.0            0.0   
             rec-188-dup-0        0.0     0.0       0.0    0.0            0.0   
             rec-334-dup-0        0.0     0.0       0.0    0.0            0.0   
             rec-469-dup-0        0.0     0.0       0.0    0.0            0.0   
             rec-350-dup-0        0.0     0.0       0.0    0.0            0.0   

                            soc_sec_id  
rec_id_1     rec_id_2                   
rec-122-org  rec-223-org             0  
rec-373-org  rec-223-org             0  
             rec-122-org             0  
rec-10-dup-0 rec-223-org             0  
             rec-122-org             0  
...                                ...  
rec-212-org  rec-327-org             0  
             rec-188-dup-0           0  
             rec-334-dup-0           0  
             rec-469-dup-0           0  
             rec-350-dup-0           0  

[499500 rows x 10 columns]

In [11]:
features_test = comp.compute(candidate_links_test, df_test)

In [12]:
features_test

given_name   surname  street_number  address_1  \
rec_id_1      rec_id_2                                                        
rec-712-dup-0 rec-2778-org     0.466667  0.455556            0.0        0.0   
rec-1321-org  rec-2778-org     0.447619  0.437037            0.0        0.0   
              rec-712-dup-0    0.000000  0.425926            0.0        0.0   
rec-3004-org  rec-2778-org     0.561905  0.577778            0.0        0.0   
              rec-712-dup-0    0.447619  0.555556            0.0        0.0   
...                                 ...       ...            ...        ...   
rec-1143-org  rec-2284-org     0.511111  0.483333            0.0        0.0   
              rec-1487-org     0.455556  0.633333            0.0        0.0   
              rec-1856-org     0.466667  0.464286            0.0        0.0   
              rec-3307-org     0.466667  0.000000            0.0        0.0   
              rec-227-org      0.447619  0.000000            0.0        0.0   

                             address_2  suburb  postcode  state  \
rec_id_1      rec_id_2                                            
rec-712-dup-0 rec-2778-org         0.0     0.0       0.0    1.0   
rec-1321-org  rec-2778-org         0.0     0.0       0.0    0.0   
              rec-712-dup-0        0.0     0.0       0.0    0.0   
rec-3004-org  rec-2778-org         0.0     0.0       0.0    0.0   
              rec-712-dup-0        0.0     0.0       0.0    0.0   
...                                ...     ...       ...    ...   
rec-1143-org  rec-2284-org         0.0     0.0       0.0    0.0   
              rec-1487-org         0.0     0.0       0.0    0.0   
              rec-1856-org         0.0     0.0       0.0    0.0   
              rec-3307-org         0.0     0.0       0.0    0.0   
              rec-227-org          0.0     0.0       0.0    0.0   

                             date_of_birth  soc_sec_id  
rec_id_1      rec_id_2                                  
rec-712-dup-0 rec-2778-org             0.0           0  
rec-1321-org  rec-2778-org             0.0           0  
              rec-712-dup-0            0.0           0  
rec-3004-org  rec-2778-org             0.0           0  
              rec-712-dup-0            0.0           0  
...                                    ...         ...  
rec-1143-org  rec-2284-org             0.0           0  
              rec-1487-org             0.0           0  
              rec-1856-org             0.0           0  
              rec-3307-org             0.0           0  
              rec-227-org              0.0           0  

[12497500 rows x 10 columns]

In [13]:
# Compute the total score from scores
# Sort them in descending order
print(features_train.sum(axis=1).value_counts().sort_index(ascending=False))

10.000000       34
9.980000         1
9.977778         2
9.971429         4
9.966667         4
             ...  
0.285185         1
0.282407         2
0.277778         1
0.275000         2
0.000000     22527
Name: count, Length: 11929, dtype: int64


In [14]:
# Classification
matches = features_train[features_train.sum(axis=1) > 7].copy()  # Only take the ones with total score above 7/10
matches["score"] = matches.loc[:, "given_name": "soc_sec_id"].sum(axis=1)  # Add in a score column
matches = matches.sort_values(by=["score"], ascending=True)  # Sort in descending order based on the score

In [15]:
matches

,,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,score
rec_id_1,rec_id_2,,,,,,,,,,,
rec-163-dup-0,rec-163-org,0.539683,0.539683,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0,7.079365
rec-344-dup-0,rec-344-org,0.000000,0.422222,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1,7.422222
rec-195-dup-0,rec-195-org,1.000000,0.422619,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0,7.422619
rec-401-org,rec-401-dup-0,1.000000,0.441667,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0,7.441667
rec-173-org,rec-173-dup-0,1.000000,0.483333,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,7.483333
...,...,...,...,...,...,...,...,...,...,...,...,...
rec-350-dup-0,rec-350-org,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,10.000000
rec-119-dup-0,rec-119-org,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,10.000000
rec-94-dup-0,rec-94-org,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,10.000000


In [16]:
df_train.loc["rec-163-dup-0"]

given_name              rickett
surname                  oliver
street_number               194
address_1          sturt afenue
address_2        shoppingcentre
suburb                jamedtown
postcode                   2256
state                       qld
date_of_birth          19431020
soc_sec_id              6090460
Name: rec-163-dup-0, dtype: object

In [17]:
df_train.loc["rec-163-org"]

given_name                oliver
surname                  rickett
street_number                194
address_1           sturt avenue
address_2        shopping centre
suburb                 jamestown
postcode                    2526
state                        qld
date_of_birth           19431020
soc_sec_id               6090360
Name: rec-163-org, dtype: object

In [18]:
# Display the classification quality
def display_quality(true_links, df):
    print(recordlinkage.precision(true_links, df))
    print(recordlinkage.recall(true_links, df))
    print(recordlinkage.fscore(true_links, df))

In [19]:
display_quality(y_train, matches)

1.0
0.86
0.924731182795699


In [21]:
pairs = features_train
true_match_labels = pairs.index.intersection(y_train)

In [22]:
log_reg = recordlinkage.LogisticRegressionClassifier()

In [23]:
log_reg.fit(pairs, true_match_labels)

log_reg_predict = log_reg.predict(features_train)
log_reg_predict_test = log_reg.predict(features_test)

display_quality(y_train, log_reg_predict)
display_quality(y_test, log_reg_predict_test)

1.0
0.998
0.998998998998999
1.0
0.9788004136504653
0.9892866475045727


In [24]:
log_reg_predict

MultiIndex([('rec-344-dup-0',   'rec-344-org'),
            (  'rec-251-org', 'rec-251-dup-0'),
            ('rec-335-dup-0',   'rec-335-org'),
            ( 'rec-23-dup-0',    'rec-23-org'),
            (  'rec-382-org', 'rec-382-dup-0'),
            ( 'rec-70-dup-0',    'rec-70-org'),
            (  'rec-291-org', 'rec-291-dup-0'),
            (  'rec-179-org', 'rec-179-dup-0'),
            ('rec-452-dup-0',   'rec-452-org'),
            (  'rec-383-org', 'rec-383-dup-0'),
            ...
            ('rec-416-dup-0',   'rec-416-org'),
            (  'rec-330-org', 'rec-330-dup-0'),
            (  'rec-385-org', 'rec-385-dup-0'),
            ('rec-113-dup-0',   'rec-113-org'),
            (  'rec-327-org', 'rec-327-dup-0'),
            ('rec-188-dup-0',   'rec-188-org'),
            ('rec-334-dup-0',   'rec-334-org'),
            ('rec-469-dup-0',   'rec-469-org'),
            ('rec-350-dup-0',   'rec-350-org'),
            (  'rec-212-org', 'rec-212-dup-0')],
           names=['rec_

In [25]:
log_reg_predict_test

MultiIndex([(  'rec-63-dup-1',   'rec-63-dup-0'),
            ( 'rec-232-dup-0',    'rec-232-org'),
            ('rec-3123-dup-0',   'rec-3123-org'),
            ('rec-2691-dup-2',   'rec-2691-org'),
            ('rec-1442-dup-2', 'rec-1442-dup-1'),
            ('rec-1665-dup-0', 'rec-1665-dup-2'),
            (  'rec-3707-org', 'rec-3707-dup-1'),
            ( 'rec-901-dup-0',    'rec-901-org'),
            (   'rec-201-org',  'rec-201-dup-2'),
            ( 'rec-201-dup-4',  'rec-201-dup-2'),
            ...
            ( 'rec-540-dup-0',    'rec-540-org'),
            ( 'rec-540-dup-0',  'rec-540-dup-2'),
            ('rec-2822-dup-0', 'rec-2822-dup-1'),
            ('rec-2822-dup-0',   'rec-2822-org'),
            ('rec-2822-dup-0', 'rec-2822-dup-2'),
            (  'rec-2889-org', 'rec-2889-dup-0'),
            (  'rec-2889-org', 'rec-2889-dup-2'),
            (  'rec-2889-org', 'rec-2889-dup-1'),
            (  'rec-1609-org', 'rec-1609-dup-0'),
            (  'rec-1609-org', 're

In [26]:
svm_model = recordlinkage.SVMClassifier()

In [28]:
svm_model.fit(pairs, true_match_labels)
svm_predict = svm_model.predict(features_train)
svm_predict_test = svm_model.predict(features_test)

display_quality(y_train, svm_predict)
display_quality(y_test, svm_predict_test)

1.0
1.0
1.0
1.0
0.9968976215098242
0.9984464008285863


In [30]:
svm_predict

MultiIndex([('rec-344-dup-0',   'rec-344-org'),
            (  'rec-251-org', 'rec-251-dup-0'),
            ('rec-335-dup-0',   'rec-335-org'),
            ( 'rec-23-dup-0',    'rec-23-org'),
            (  'rec-382-org', 'rec-382-dup-0'),
            ( 'rec-70-dup-0',    'rec-70-org'),
            (  'rec-291-org', 'rec-291-dup-0'),
            (  'rec-179-org', 'rec-179-dup-0'),
            ('rec-452-dup-0',   'rec-452-org'),
            (  'rec-383-org', 'rec-383-dup-0'),
            ...
            ('rec-416-dup-0',   'rec-416-org'),
            (  'rec-330-org', 'rec-330-dup-0'),
            (  'rec-385-org', 'rec-385-dup-0'),
            ('rec-113-dup-0',   'rec-113-org'),
            (  'rec-327-org', 'rec-327-dup-0'),
            ('rec-188-dup-0',   'rec-188-org'),
            ('rec-334-dup-0',   'rec-334-org'),
            ('rec-469-dup-0',   'rec-469-org'),
            ('rec-350-dup-0',   'rec-350-org'),
            (  'rec-212-org', 'rec-212-dup-0')],
           names=['rec_

In [31]:
svm_predict_test

MultiIndex([(  'rec-63-dup-1',   'rec-63-dup-0'),
            ( 'rec-232-dup-0',    'rec-232-org'),
            ('rec-3123-dup-0',   'rec-3123-org'),
            ('rec-2691-dup-2',   'rec-2691-org'),
            ('rec-1442-dup-2', 'rec-1442-dup-1'),
            ('rec-1665-dup-0', 'rec-1665-dup-2'),
            (  'rec-3707-org', 'rec-3707-dup-1'),
            ( 'rec-901-dup-0',    'rec-901-org'),
            (   'rec-201-org',  'rec-201-dup-2'),
            ( 'rec-201-dup-4',  'rec-201-dup-2'),
            ...
            ( 'rec-540-dup-0',    'rec-540-org'),
            ( 'rec-540-dup-0',  'rec-540-dup-2'),
            ('rec-2822-dup-0', 'rec-2822-dup-1'),
            ('rec-2822-dup-0',   'rec-2822-org'),
            ('rec-2822-dup-0', 'rec-2822-dup-2'),
            (  'rec-2889-org', 'rec-2889-dup-0'),
            (  'rec-2889-org', 'rec-2889-dup-2'),
            (  'rec-2889-org', 'rec-2889-dup-1'),
            (  'rec-1609-org', 'rec-1609-dup-0'),
            (  'rec-1609-org', 're